# Ownership

## [What Is Ownership](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#what-is-ownership)

Now time to learn about ownership structures in Rust, and how it manages memory.

I've heard this is both an interesting and difficult feature of the language.

So it says they use a system of ownership rules as a kind of "third option" to avoid both explicit freeing and garbage collection.

They're really setting my expectations for something unique and difficult to understand.

I'm ready!😄 

Hmm...

## [The Stack and the Heap](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#the-stack-and-the-heap)

So this doesn't really tell my anything I don't already know.

The various mechanisms of stack and heap allocations and their trade off are described.

But those are already well and truely drummed into me though programming education and experience.

[It's basically like this...](https://www.smbc-comics.com/?id=3892)

(sorry)



## [Ownership Rules](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#ownership-rules)

I see. So every value (object?) has some object that "owns" it exclusively. 

And the scope it tracked and it gets deleted if it goes out of scope.

Well that's sounding like local variables thus far.

Let's see if this Rust Jupyter kernel is working.


In [4]:
{
    let s = "hello";
    println!("{}", s);
}
// Now she's dead and buried fellas. The world's coolest S.

hello


()

This is all automatic heap allocation right?


## [The String Type](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#the-string-type)
Well, it's time to explore the inconsistant behaviour of this type.

So what makes strings different from string literals?

`println!()` treats them like they're two different things, which is quite unusual.

This is apparently related to ownership?

So a literal is hardcoded, but a `String` could be mutable (at least if the type is).

```rust
let s = String::from("hello");
```

So this is a translation between a literal and a string. 
 * It's done explicitly in Rust rather than automatically like most languages?
 * The `::from()` syntax implies overloading.
 * Memory to automatically allocated on the heap while it's in scope.
 * The string/literal split seems to be for the sake of efficiency and memory management.
 

 

## [Memory and Allocation](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#memory-and-allocation)
The memory manager will automatically call `drop` when the scope is over.

I have some... questions about this.
 * What if we want to use a variable after to scope is finished?
    * Does it get copied to a higher scope somehow?
 * Isn't this complicated by the automatic creation of shadow variables?
    * No, that has to be done explicitly with `let`.
 * How are function returns then passed back without losing them?
 * Just happy not to have manual memory management.

## [Variables and Data Interaction with Move](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#variables-and-data-interacting-with-move)

So they showed this code:
```rust
let x = 5;
let y = x;
let s1 = String::from("hello");
let s2 = s1;
```

The first two variable have their own independent values and the second two both point to the same String.

The tutorial seemed to thing this would be unintuitive, but it's an entirely expected result with pointers.

I would be more surprise if they actually did point to different objects.

I'm not sure what they're getting at.

```rust
    let s1 = String::from("hello");
    let s2 = s1;

    println!("{s1}, world!");
```

Oh. `s1` is now invalid because *`s2` has taken ownership of the memory region*.

Following the initial rules there can only be one! One owner that it.

Well, that is an unintuive result.

Normally you'd just have two pointers (or references) to the same thing, and... problems.

Now I'm interested.



## [Variables and Data Interacting with Clone](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#variables-and-data-interacting-with-clone)
They do have a `.clone()` keyword if you do want a deep copy.

But this copies all the memory and gives it to another pointer.

You still have exactly one unique memory region per pointer.

```rust
let jango_fett = String::from("hello");
let boba_fett = s1.clone();
```

Your normal "by value" types do not follow these rules. 
 * These types actually are copied when passed to different variables.
 * This includes ints, bools, floats, chars.
 * Tuples are a little unexpected.
    * They implicitly copied, if they contain only the simple scalar types above.
    * If they contain a `String` or other reference type they are *not* implicitly copied but have ownership tranferred.


## [Ownership and Functions](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#ownership-and-functions)

Now this snippet is even more unexpected:

```rust
    let s = String::from("hello");  // s comes into scope

    takes_ownership(s);             // s's value moves into the function...
                                    // ... and so is no longer valid here
...
fn takes_ownership(some_string: String) { // some_string comes into scope
    println!("{some_string}");
} // Here, some_string goes out of scope and `drop` is called. The backing
  // memory is freed.

```

But it's the logical result of our afforementioned rules.

The function "steals" the reference from us.

But then... it a simple print function can do that how can we run any function on a pointer???

Is there a flag to return ownership?

## [Return Values and Scope](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#return-values-and-scope)

Hmm...

```rust
fn main() {
    let s1 = gives_ownership();         // gives_ownership moves its return
                                        // value into s1

    let s2 = String::from("hello");     // s2 comes into scope

    let s3 = takes_and_gives_back(s2);  // s2 is moved into
                                        // takes_and_gives_back, which also
                                        // moves its return value into s3
} // Here, s3 goes out of scope and is dropped. s2 was moved, so nothing
  // happens. s1 goes out of scope and is dropped.

fn gives_ownership() -> String {             // gives_ownership will move its
                                             // return value into the function
                                             // that calls it

    let some_string = String::from("yours"); // some_string comes into scope

    some_string                              // some_string is returned and
                                             // moves out to the calling
                                             // function
}

// This function takes a String and returns one
fn takes_and_gives_back(a_string: String) -> String { // a_string comes into
                                                      // scope

    a_string  // a_string is returned and moves out to the calling function
}
```
Aaaaaaaaaagh!

Don't make me code like that.

Come on, command query separation... or something.


```rust
let (s2, len) = calculate_length(s1);
```
So there's them (single-reference-only) tuples again.



## [References and Borrowing](https://doc.rust-lang.org/book/ch04-02-references-and-borrowing.html)

Apparently you can (sort of) get around this problem.

You have to pass a reference to the pointer rather than the pointer itself.
```rust
    let len = calculate_length(&s1);
...
fn calculate_length(s: &String) -> usize {
    s.len()
}
```
Is this why the APIs we've used all seem to need what you'd call "double pointers"?

Those are fairly rare in C-style languages, usually only if you want to change the pointer.

So curiously `s.len()` does *not* use an astrix \* before it.

Perhaps that's because of the function signature?

Our "reference" disappears after finishing the function, but the original still exists.

The function never owned it.

Well it seems like an acceptable compromise.


## [Mutable References](https://doc.rust-lang.org/book/ch04-01-what-is-ownership.html#return-values-and-scope)
Like `AudioPlayer`? 😆

Anyway...
```rust
change(&mut s);

fn change(some_string: &mut String) {
    some_string.push_str(", world");
}
```

This looks like another interesting case. 

So function can change it from the reference?

There's a distict lack of... extraneous operators. I nearly expected ten more symbols. 

Again, I love that mutability has to be explicitly marked and not the other way around like in many languages.

It's an output argument, but this form is probably better than the alternative.

```rust
    let mut s = String::from("hello");
    let r1 = &mut s;
    let r2 = &mut s;
```
No more than one mutable reference in the scope, single ownership rule.

The tutorial points out this also prevents many race conditions. Sounds good!


You can create mutable references within inner scopes:
```rust
    {
        let r1 = &mut s;
    } // r1 goes out of scope here, so we can make a new reference with no problems.

    let r2 = &mut s;
```

And (unexpectedly) you can create multiple immutable references.
But if any **mutable** references have to be the only one.

This is because a function using an immutable reference wouldn't expect the underlying data to change.

```rust
    let r1 = &s; // no problem
    let r2 = &s; // no problem
    let r3 = &mut s; // BIG PROBLEM
    println!("{}, {}, and {}", r1, r2, r3);
```
The final line should cause an error.

Hmm... well this is certainly tricky.

I'll have to write a few programs to get used to it!

The system seems pretty safe, unless an external thread or hardware device has access to the memory.

I wonder if there's an equivalent to `volitle` so memory mapped devices?

Well there's some OS dev flashbacks...

One final curveball.
Apparently this is okay:

```rust
    let mut s = String::from("hello");

    let r1 = &s; // no problem
    let r2 = &s; // no problem
    println!("{r1} and {r2}");
    // variables r1 and r2 will not be used after this point

    let r3 = &mut s; // no problem
    println!("{r3}");
```

Because the references are freed *after their final usage*.

I had assumed it was the end of the scope?

Not so. That could get confusing.

Anyway, the compiler should point out any errors here.






## [Dangling References](https://doc.rust-lang.org/book/ch04-02-references-and-borrowing.html#dangling-references)
Unlike the previous series of compiler errors this one is entirely expected.

```rust
fn main() {
    let reference_to_nothing = dangle();
}

fn dangle() -> &String {
    let s = String::from("hello");

    &s
}
```

It won't compile because it passes a reference to an object that'll go out of scope.

The object would now have no owner (violating the single owner rule) and the reference would be invalid.

Fortunately, the compile won't allow this kind of dangerous behaviour.

The error tells you that you probably want to pass the original value (ownership).

This is a great language! 😍




I'll go over the rules again so I remember:
 * Each object must have one variable owning it and no more than one.
     * Passing the it to another variable or function transfers ownership.
        * The original is now invalid!
     * If ownership is not transferred, it is deleted after the last reference to it within the scope.
 * An object can have any number of immutable references to it.
     * Handy for passing to functions.
     * References do not transfer ownership.
 * Any mutable reference to an object must be the *only* reference to an object.
    * This means no immutable references if a mutable exists.


I didn't have much idea on how to test this, but I put together a basic program to test this out that adds input to a string.
